# **Libraries**
***

Import needed common libraries

In [36]:
import pandas as pd 
import numpy as np 
import sqlite3

import os
print(os.listdir('../BD00-Datasets'))

['csv-datasets', 'db-datasets', 'New folder', '__MACOSX']


# **Data retrieval**
***

Construct helper function for data connection and queries

In [37]:
#conn = sqlite3.connect('../dataset/chinook.db')
#c = conn.cursor()
def run_query(q):
    with sqlite3.connect('../BD00-Datasets/db-datasets/chinook.db') as conn:
        return pd.read_sql_query(q, conn)

# **Data inspections**
***

View data structures

Construct needed data into dataset

In [38]:
# inspecting data tables
#run_query('''SELECT Name,type FROM sqlite_master WHERE type='table' ''')
tables = run_query('''SELECT Name,type FROM sqlite_master WHERE type='table' ''')
tables

,name,type
0,Album,table
1,Artist,table
2,Customer,table
3,Employee,table
4,Genre,table
5,Invoice,table
6,InvoiceLine,table
7,MediaType,table
8,Playlist,table
9,PlaylistTrack,table


In [39]:
#run_query('''SELECT * FROM Invoice''').info()
q = '''SELECT * FROM Invoice'''
invoice = run_query(q)

invoice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412 entries, 0 to 411
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   InvoiceId          412 non-null    int64  
 1   CustomerId         412 non-null    int64  
 2   InvoiceDate        412 non-null    object 
 3   BillingAddress     412 non-null    object 
 4   BillingCity        412 non-null    object 
 5   BillingState       210 non-null    object 
 6   BillingCountry     412 non-null    object 
 7   BillingPostalCode  384 non-null    object 
 8   Total              412 non-null    float64
dtypes: float64(1), int64(2), object(6)
memory usage: 29.1+ KB


In [40]:
#run_query('''SELECT * FROM InvoiceLine''').info()
q = '''SELECT * FROM InvoiceLine'''
invoice_line = run_query(q)

invoice_line.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   InvoiceLineId  2240 non-null   int64  
 1   InvoiceId      2240 non-null   int64  
 2   TrackId        2240 non-null   int64  
 3   UnitPrice      2240 non-null   float64
 4   Quantity       2240 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 87.6 KB


q = '''SELECT InvoiceLine.InvoiceId AS InvoiceNo, 
              Track.Name AS Tracks,
              InvoiceLine.Quantity 
       FROM InvoiceLine
       JOIN track 
            ON InvoiceLine.TrackId = Track.TrackId
    '''
df = run_query(q)
df

In [41]:
q = '''SELECT InvoiceLine.InvoiceId AS InvoiceNo, 
              InvoiceLine.TrackId AS TrackID,
              Track.Name AS Tracks,
              Genre.GenreId AS GenreID,
              Genre.Name AS Genres,
              InvoiceLine.Quantity,
              InvoiceLine.UnitPrice,
              Invoice.InvoiceDate,
              Invoice.CustomerId AS CustomerID,
              Invoice.BillingCountry AS Country
       FROM InvoiceLine
       JOIN Invoice
            ON InvoiceLine.InvoiceId = Invoice.InvoiceId
       JOIN track 
            ON InvoiceLine.TrackId = Track.TrackId
       JOIN genre 
            ON Track.GenreId = Genre.GenreId
    '''
df = run_query(q)
df

,InvoiceNo,TrackID,Tracks,GenreID,Genres,Quantity,UnitPrice,InvoiceDate,CustomerID,Country
0,1,2,Balls to the Wall,1,Rock,1,0.99,2009-01-01 00:00:00,2,Germany
1,1,4,Restless and Wild,1,Rock,1,0.99,2009-01-01 00:00:00,2,Germany
2,2,6,Put The Finger On You,1,Rock,1,0.99,2009-01-02 00:00:00,4,Norway
3,2,8,Inject The Venom,1,Rock,1,0.99,2009-01-02 00:00:00,4,Norway
4,2,10,Evil Walks,1,Rock,1,0.99,2009-01-02 00:00:00,4,Norway
...,...,...,...,...,...,...,...,...,...,...
2235,411,3136,Looking For Love,3,Metal,1,0.99,2013-12-14 00:00:00,44,Finland
2236,411,3145,Sweet Lady Luck,3,Metal,1,0.99,2013-12-14 00:00:00,44,Finland
2237,411,3154,Feirinha da Pavuna/Luz do Repente/Bagaço da La...,7,Latin,1,0.99,2013-12-14 00:00:00,44,Finland
2238,411,3163,Samba pras moças,7,Latin,1,0.99,2013-12-14 00:00:00,44,Finland


# **Data preparation**
***

One-Hot encode data

In [42]:
#from mlxtend.preprocessing import TransactionEncoder
#te = TransactionEncoder()
#data = te.fit(basket).transform(basket)
#data = pd.DataFrame(data, columns = te.columns_)
#data

basket = (df.groupby(['InvoiceNo', 'Genres'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))
basket

Genres,Alternative,Alternative & Punk,Blues,Bossa Nova,Classical,Comedy,Drama,Easy Listening,Electronica/Dance,Heavy Metal,...,Pop,R&B/Soul,Reggae,Rock,Rock And Roll,Sci Fi & Fantasy,Science Fiction,Soundtrack,TV Shows,World
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets

Genres,Alternative,Alternative & Punk,Blues,Bossa Nova,Classical,Comedy,Drama,Easy Listening,Electronica/Dance,Heavy Metal,...,Pop,R&B/Soul,Reggae,Rock,Rock And Roll,Sci Fi & Fantasy,Science Fiction,Soundtrack,TV Shows,World
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
409,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
410,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


# **Data modeling**
***

Classify data into itemsets

In [44]:
# Import modeling libraries
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules

In [45]:
frequent_itemsets = apriori(basket_sets, min_support=0.005, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.009709,(Alternative)
1,0.225728,(Alternative & Punk)
2,0.065534,(Blues)
3,0.016990,(Bossa Nova)
4,0.036408,(Classical)
...,...,...
191,0.007282,"(Rock, Latin, Reggae, Soundtrack)"
192,0.007282,"(Pop, Rock, Reggae, Soundtrack)"
193,0.009709,"(Rock, Latin, Soundtrack, Easy Listening, Alte..."
194,0.007282,"(Rock, Rock And Roll, Jazz, Metal, Alternative..."


In [46]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.009709,(Alternative),1
1,0.225728,(Alternative & Punk),1
2,0.065534,(Blues),1
3,0.016990,(Bossa Nova),1
4,0.036408,(Classical),1
...,...,...,...
191,0.007282,"(Rock, Latin, Reggae, Soundtrack)",4
192,0.007282,"(Pop, Rock, Reggae, Soundtrack)",4
193,0.009709,"(Rock, Latin, Soundtrack, Easy Listening, Alte...",5
194,0.007282,"(Rock, Rock And Roll, Jazz, Metal, Alternative...",5


In [47]:
frequent_itemsets[ (frequent_itemsets['length'] >= 2) &
                   (frequent_itemsets['support'] >= 0.005) ]

,support,itemsets,length
24,0.007282,"(Alternative, Classical)",2
25,0.019417,"(Blues, Alternative & Punk)",2
26,0.009709,"(Easy Listening, Alternative & Punk)",2
27,0.007282,"(Alternative & Punk, Electronica/Dance)",2
28,0.007282,"(Hip Hop/Rap, Alternative & Punk)",2
...,...,...,...
191,0.007282,"(Rock, Latin, Reggae, Soundtrack)",4
192,0.007282,"(Pop, Rock, Reggae, Soundtrack)",4
193,0.009709,"(Rock, Latin, Soundtrack, Easy Listening, Alte...",5
194,0.007282,"(Rock, Rock And Roll, Jazz, Metal, Alternative...",5


# **Calculation**
***

Build rules for calculation

In [48]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Alternative),(Classical),0.009709,0.036408,0.007282,0.750000,20.600000,0.006928,3.854369
1,(Classical),(Alternative),0.036408,0.009709,0.007282,0.200000,20.600000,0.006928,1.237864
2,(Blues),(Alternative & Punk),0.065534,0.225728,0.019417,0.296296,1.312624,0.004625,1.100281
3,(Alternative & Punk),(Blues),0.225728,0.065534,0.019417,0.086022,1.312624,0.004625,1.022416
4,(Easy Listening),(Alternative & Punk),0.009709,0.225728,0.009709,1.000000,4.430108,0.007517,inf
...,...,...,...,...,...,...,...,...,...
973,(Rock),"(Reggae, Latin, Soundtrack, Pop)",0.524272,0.007282,0.007282,0.013889,1.907407,0.003464,1.006700
974,(Latin),"(Reggae, Rock, Soundtrack, Pop)",0.283981,0.007282,0.007282,0.025641,3.521368,0.005214,1.018843
975,(Pop),"(Rock, Latin, Reggae, Soundtrack)",0.031553,0.007282,0.007282,0.230769,31.692308,0.007052,1.290534
976,(Soundtrack),"(Rock, Latin, Reggae, Pop)",0.029126,0.007282,0.007282,0.250000,34.333333,0.007069,1.323625


In [49]:
rules[ (rules['lift'] >= 12) 
     & (rules['confidence'] >= 0.79) 
     & (rules['confidence'] != 1) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
25,(Heavy Metal),(Blues),0.012136,0.065534,0.009709,0.800,12.207407,0.008913,4.672330
261,"(Metal, Heavy Metal)",(Blues),0.012136,0.065534,0.009709,0.800,12.207407,0.008913,4.672330
265,(Heavy Metal),"(Metal, Blues)",0.012136,0.021845,0.009709,0.800,36.622222,0.009444,4.890777
267,"(Rock, Heavy Metal)",(Blues),0.012136,0.065534,0.009709,0.800,12.207407,0.008913,4.672330
271,(Heavy Metal),"(Rock, Blues)",0.012136,0.031553,0.009709,0.800,25.353846,0.009326,4.842233
630,"(Metal, Jazz, Alternative & Punk)",(Rock And Roll),0.012136,0.009709,0.009709,0.800,82.400000,0.009591,4.951456
711,"(Metal, Rock, Heavy Metal)",(Blues),0.012136,0.065534,0.009709,0.800,12.207407,0.008913,4.672330
716,"(Metal, Heavy Metal)","(Rock, Blues)",0.012136,0.031553,0.009709,0.800,25.353846,0.009326,4.842233
718,"(Rock, Heavy Metal)","(Metal, Blues)",0.012136,0.021845,0.009709,0.800,36.622222,0.009444,4.890777
723,(Heavy Metal),"(Metal, Rock, Blues)",0.012136,0.014563,0.009709,0.800,54.933333,0.009532,4.927184


In [50]:
print(f"Heavy Metal : {basket['Heavy Metal'].sum()} sold"'\n'f"Blues : {basket['Blues'].sum()} sold")
rules.loc[[25]]

Heavy Metal : 12.0 sold
Blues : 61.0 sold


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
25,(Heavy Metal),(Blues),0.012136,0.065534,0.009709,0.8,12.207407,0.008913,4.67233


In [51]:
print( f"World : {basket['World'].sum()} sold"'\n'f"Rock  : {basket['Rock'].sum()} sold"'\n' + 
       f"Metal : {basket['Metal'].sum()} sold"'\n'f"Latin : {basket['Latin'].sum()} sold" )
rules.loc[[813]]

World : 13.0 sold
Rock  : 835.0 sold
Metal : 264.0 sold
Latin : 386.0 sold
